<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection**

In [ ]:
#Scrapping works well with stuctured data, like tables etc.
#Also, there is the issue of labelling:
#Using myself or someelse may result in bias (interpretation of swahili words or sentences based on given labels)
  #Also, this will be tedious and costly (time)

In [ ]:
#Any other alternative???